# Rudimentary Flight Path Plot

Set the ICAO address for a tracked aircraft, with position information, and the time range (taken from records in the POSITION table) in the next cell

In [293]:
from datetime import datetime

aircraft_address = "44034B"
start_timestamp = datetime(2025, 10, 15, 19, 40, 15)
end_timestamp = None

In [294]:

EARTH_RADIUS = 6371000.0

MAX_SEGMENT_GAP_SECONDS = 90

TITLE = dict(
    x=0.5,                  # horizontal centre (0 = left, 0.5 = middle, 1 = right)
    xanchor='center',       # anchor the title to the centre position
    yanchor='top',          # keep it above the chart
    font=dict(size=18)      # optional: make it a bit larger
)

COLOURBAR = dict(
    title="Altitude (m)",
    x=1.05,
    xanchor="left",
    y=0.5
)

LIGHTING = dict(
    ambient=0.6,     # general scene light
    diffuse=0.8,     # brightness on lit surfaces
    specular=0.3,    # shininess
    roughness=0.5,   # 0 = mirror smooth, 1 = matte
    fresnel=0.2      # reflectivity edge highlight
)

LIGHTPOSITION = dict(
    x=2000,          # east-west offset in local coords
    y=0,             # north-south offset
    z=8000           # altitude of the light source (sun height)
)

LEGEND = dict(
    x=0.02,
    y=0.98, 
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="rgba(0,0,0,0.2)",
    borderwidth=1
)

In [295]:
%run pathutils.ipynb
%run database.ipynb

In [296]:
import pandas as pd

def load_flight_path(address, from_timestamp=None, to_timestamp=None):
    # Construct and run the query for the specified address
    query = construct_query("tracker", "reports", "list-flight-path.sql", { "ADDRESS": address })
    df = query_data("tracker", query)

    # Convert the timestamp to datetime and filter based on the timestamp limits
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce", utc=False)
    df = df.dropna(subset=["Latitude","Longitude","Altitude","Distance","Timestamp"]).reset_index(drop=True)

    if from_timestamp:
        df = df[(df["Timestamp"] > from_timestamp)]

    if to_timestamp:
        df = df[(df["timestamp"] < to_timestamp)]

    # Convert altitude to metres and make sure all properties are numeric
    df["Altitude"] = pd.to_numeric(df["Altitude"], errors="coerce") * 0.3048
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")
    df["Distance"] = pd.to_numeric(df["Distance"], errors="coerce")

    return df

In [297]:
import numpy as np

def calculate_z_range(flight_path_df):
    """
    Calculate a padded Z-range based on the minimum and maximum altitude in the whole plot
    """

    # Calculate the range
    altitude = flight_path_df["Altitude"].to_numpy()
    zmin = float(np.nanmin(altitude))
    zmax = float(np.nanmax(altitude))

    # Pad it (10% or a minimum of 250m padding)
    zpad = max((zmax - zmin) * 0.1, 250.0)
    zmin = max(0.0, zmin - zpad)
    zmax = zmax + zpad

    return zmin, zmax

In [298]:
def segment_on_gaps(df):
    """
    Split into segments at points where the time gaps exceed max_gap_seconds
    """
    s = df.sort_values("Timestamp").reset_index(drop=True).copy()
    dt = s["Timestamp"].diff().dt.total_seconds().fillna(0)
    group_id = (dt > MAX_SEGMENT_GAP_SECONDS).cumsum()
    return [seg.reset_index(drop=True) for _, seg in s.groupby(group_id)]

In [299]:
import numpy as np

def coordinates_to_local_xy(latitude, longitude):
    """
    Equirectangular (plate carrée) projection - convert latitude and longitude into east-west (x)
    and north-south (y). x increases Eastwards and y increases Northwards relative to a reference
    point, taken as the first point in the trace
    """
    latitude = np.asarray(latitude)
    longitude = np.asarray(longitude)

    # Convert the points to radians
    latitude_rad = np.radians(latitude)
    longitude_rad = np.radians(longitude)
    ref_latitude_rad = np.radians(latitude[0])
    ref_longitude_rad = np.radians(longitude[0])

    # Calculate the differences between the latitude and the reference point
    delta_longitude = longitude_rad - ref_longitude_rad
    delta_latitude = latitude_rad - ref_latitude_rad

    # Calculate to local distance in meters: x = East-West, y = North-South
    x = delta_longitude * np.cos(ref_latitude_rad) * EARTH_RADIUS   
    y = delta_latitude * EARTH_RADIUS

    return x, y

In [300]:
import numpy as np

def curtain_mesh(x, y, z):
    """
    Build a vertical 'curtain' mesh under a polyline (x,y,z). For every point in the flight, the top vertex
    is the position of the aircraft and the bottom vertex is the same (x, y) but on the ground. Between
    consecutive segments, the vertices are joined by triangles

    Returns:
        X, Y, Z: vertex coordinates (each length 2*len(x))
        I, J, K: triangle index arrays for Plotly Mesh3d

    """
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.asarray(z)

    # Allocate space for top and bottom vertices
    X = np.empty(2*len(x))
    Y = np.empty(2*len(y))
    Z = np.empty(2*len(z))

    # For each point, make a top vertex (even index) and a bottom vertex (odd index)
    X[0::2], X[1::2] = x, x
    Y[0::2], Y[1::2] = y, y
    Z[0::2], Z[1::2] = z, 0.0

    i=j=k=[]
    i=[]
    j=[]
    k=[]

    # Build triangles
    for t in range(len(x)-1):
        # indices of the four vertices forming one quad
        t0 = 2*t            # top of point t
        b0 = 2*t + 1        # bottom of point t
        t1 = 2*(t+1)        # top of point t+1
        b1 = 2*(t+1)+1      # bottom of point t+1

        # Split the quad into two triangles: (t0, b0, b1) and (t0, b1, t1)
        i += [t0, t0]
        j += [b0, b1]
        k += [b1, t1]

    return X, Y, Z, np.array(i), np.array(j), np.array(k)

In [301]:
import numpy as np
import plotly.graph_objects as go

def plot_flight_ribbon(df, zmin, zmax):
    # Calculate the east-west, north-south coordinates and the curtain mesh
    x, y = coordinates_to_local_xy(df["Latitude"], df["Longitude"])
    X, Y, Z, I, J, K = curtain_mesh(x, y, df["Altitude"])

    # Create a new figure
    fig = go.Figure()

    # Add the ribbon to the figure
    fig.add_trace(
        go.Mesh3d(
            x=X, y=Y, z=Z,
            i=I, j=J, k=K,
            opacity=0.85,
            flatshading=True,
            name="Ribbon",
            intensity=Z,
            colorscale="Plasma",
            colorbar=COLOURBAR,
            lighting=LIGHTING,
            lightposition=LIGHTPOSITION,
            showscale=True
        )
    )

    # Add the flight path and ground trace
    fig.add_trace(go.Scatter3d(x=x, y=y, z=df["Altitude"], mode="lines", line=dict(width=4), name="Flight path"))
    fig.add_trace(go.Scatter3d(x=x, y=y, z=np.zeros_like(df["Altitude"]), mode="lines", line=dict(width=2, dash="dash"), name="Ground trace"))

    # Apply z-range explicitly and turn off autorange
    fig.update_scenes(aspectmode="data", zaxis_autorange=False, zaxis_range=[zmin, zmax])

    return fig

In [302]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import math

def attach_info_strip(fig_3d, info, columns=3, row_heights=(0.82, 0.18), height=820):
    """
    Takes an existing 3D figure (with a single scene) and returns a new figure
    that has the 3D scene on row 1 and a Table 'info strip' on row 2.

    info: dict like {"Flight": "BA123 (SPEEDBIRD)", "Airline": "British Airways", ...}
    columns: how many items per line in the strip.
    """
    # Create a 2-row figure: top = 3D scene, bottom = domain for Table
    fig = make_subplots(
        rows=2, cols=1,
        specs=[[{"type": "scene"}],
               [{"type": "domain"}]],
        row_heights=list(row_heights),
        vertical_spacing=0.06
    )

    # Move all traces from the original fig into row 1
    for tr in fig_3d.data:
        fig.add_trace(tr, row=1, col=1)

    # --- copy layout WITHOUT overriding the subplot's scene domain ---
    if getattr(fig_3d.layout, "scene", None) is not None:
        scene_json = fig_3d.layout.scene.to_plotly_json()
        scene_json.pop("domain", None)          # <-- CRITICAL: don't clobber subplot domain
        fig.update_layout(scene=scene_json)

    # keep your title/legend/coloraxis if present
    if getattr(fig_3d.layout, "title", None) is not None:
        fig.update_layout(title=fig_3d.layout.title)
    if getattr(fig_3d.layout, "legend", None) is not None:
        fig.update_layout(legend=fig_3d.layout.legend)
    if getattr(fig_3d.layout, "coloraxis", None) is not None:
        fig.update_layout(coloraxis=fig_3d.layout.coloraxis)

    # ensure figure is tall enough and margins leave room
    fig.update_layout(
        height=height,
        margin=dict(l=60, r=60, t=70, b=20)
    )

    # Build a compact table from the info dict
    pairs = [f"<b>{k}</b>  {v}" for k, v in info.items()]

    rows_needed = math.ceil(len(pairs) / columns)
    table_rows = []
    for r in range(rows_needed):
        row_items = pairs[r*columns:(r+1)*columns]
        # pad to full width for a tidy grid
        row_items += [""] * (columns - len(row_items))
        table_rows.append(row_items)

    # Transpose for go.Table (values is list-of-columns)
    cols = list(map(list, zip(*table_rows)))

    fig.add_trace(
        go.Table(
            header=dict(
                values=[""] * columns,
                line=dict(width=0),
                fill_color="rgba(0,0,0,0)",
                height=8
            ),
            cells=dict(
                values=cols,
                align="left",
                height=22,
                line=dict(color="rgba(0,0,0,0.08)", width=1),
                fill_color=[["rgba(255,255,255,0.0)"] * rows_needed]*columns
            )
        ),
        row=2, col=1
    )

    # Optional: a faint border box feel for the strip via background rectangle
    fig.update_layout(
        annotations=list(fig.layout.annotations) + [
            dict(
                xref="paper", yref="paper",
                x=0.0, y=0.0, xanchor="left", yanchor="bottom",
                text="", showarrow=False,
                bgcolor="rgba(255,255,255,0.65)",
                bordercolor="rgba(0,0,0,0.15)", borderwidth=1,
                xshift=10, yshift=100
            )
        ]
    )

    return fig

In [303]:
def save_html(fig, aircraft_address):
    """
    Export the plot to an interactive HTML file
    """
    export_path = get_export_folder_path()
    path = export_path / f"flight_path_{aircraft_address}.html"
    fig.write_html(path, include_plotlyjs="cdn")
    return path

In [304]:
# Load the flight path data and calculate the altitude limits
df = load_flight_path(aircraft_address, start_timestamp, end_timestamp)
zmin, zmax = calculate_z_range(df)

# Get the data for the info panel
information = {
    "Callsign": df["Callsign"].iloc[0],
    "Registration": df["Registration"].iloc[0],
    "Model":  df["Model"].iloc[0],
    "Flight": df["FlightIATA"].iloc[0],
    "Airline": df["AirlineName"].iloc[0],
    "Route": df["Route"].iloc[0],
    "Start": df["Timestamp"].min().strftime("%Y-%m-%d %H:%M:%S"),
    "End": df["Timestamp"].max().strftime("%Y-%m-%d %H:%M:%S")
}

# add_info_panel(fig, **info_kwargs)
callsign = df["Callsign"].iloc[0]
flight_number = df["FlightIATA"].iloc[0]

# Construct the title
title = f"Flight Path for Aircraft {aircraft_address}"

# Segment to avoid long straight connectors across time gaps
segments = segment_on_gaps(df)

# Create a plot and set the title and legend properties
fig = go.Figure()
fig.update_layout(
    title=dict(
        text=title,
        x=0.5,                  # horizontal centre
        xanchor='center',       # anchor the title to the centre position
        yanchor='top',          # keep it above the chart
        font=dict(size=18)      # optional: make it a bit larger
    ),
    legend=LEGEND
)

# Build a single figure and add each segment as its own ribbon
for idx, seg in enumerate(segments, 1):
    if len(seg) > 2: 
        title = f"{seg["Callsign"].iloc[0] or aircraft_address} — segment {idx}"
        f = plot_flight_ribbon(seg.rename(columns={"Timestamp": "time"}), zmin, zmax)

        # Merge traces from f into fig
        for tr in f.data:
            fig.add_trace(tr)

# Set Z-scaling and show the plot
fig.update_scenes(zaxis_autorange=False, zaxis_range=[float(zmin), float(zmax)])

# Attach the info strip
final_fig = attach_info_strip(fig, information, columns=3)

# Save to interactive HTML
save_html(final_fig, aircraft_address)

# Show the figure
final_fig.show()